In [1]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold 
from sklearn.ensemble import RandomForestClassifier


In [3]:
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb

In [4]:
from sklearn.linear_model import (SGDClassifier, LogisticRegression)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [5]:
from sklearn.model_selection import GridSearchCV
import numpy as np

In [6]:
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [7]:
#чтение датасета
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
PassengerId = test['PassengerId']
train = train.drop(['Cabin', 'Ticket', 'Name', 'PassengerId'], axis=1)
test = test.drop(['Cabin', 'Ticket', 'Name', 'PassengerId'], axis=1)

In [8]:
train.describe()



,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
test.describe()

,Pclass,Age,SibSp,Parch,Fare
count,418.000000,332.000000,418.000000,418.000000,417.000000
mean,2.265550,30.272590,0.447368,0.392344,35.627188
std,0.841838,14.181209,0.896760,0.981429,55.907576
min,1.000000,0.170000,0.000000,0.000000,0.000000
25%,1.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,27.000000,0.000000,0.000000,14.454200
75%,3.000000,39.000000,1.000000,0.000000,31.500000
max,3.000000,76.000000,8.000000,9.000000,512.329200


In [10]:
#feature engineering
train_plus_test = [train, test]
#заполнение пропущенных значений средними, создание новых признаков, кодировка категориальных признаков
for dataset in train_plus_test:
    dataset['Age'] = dataset['Age'].fillna(dataset['Age'].mean()) 
    dataset['Embarked'] = dataset['Embarked'].fillna('S')  #большая часть пассажиров зашла на борт в Southampton 
    dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].mean())
    dataset['Sex'] = dataset['Sex'].map({'male': 0, 'female': 1}).astype(int) 
    dataset['Embarked'] = dataset['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int) 
    dataset['Family_size'] = dataset['SibSp'] + dataset['Parch'] 

In [11]:
#разбиение данных колонки Age на 5 категорий
for dataset in train_plus_test:
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1 
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64, 'Age'] = 4
    dataset['Age'] = dataset['Age'].astype(int)


In [12]:
#разбиение данных колонки Fare на 5 категорий
for dataset in train_plus_test:
    dataset.loc[dataset['Fare'] <= 7.9, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.9) & (dataset['Fare'] <= 14.5), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.5) & (dataset['Fare'] <= 31.2), 'Fare'] = 2
    dataset.loc[dataset['Fare'] > 31.2 , 'Fare'] = 2
    dataset['Fare'] = dataset['Fare'].astype(int)

In [13]:
#содание нового категориального признака - путешествовал ли пассажир один
for dataset in train_plus_test:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['Family_size'] >= 1, 'IsAlone'] = 1

In [14]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family_size,IsAlone
0,0,3,0,1,1,0,0,0,1,1
1,1,1,1,2,1,0,2,1,1,1
2,1,3,1,1,0,0,1,0,0,0
3,1,1,1,2,1,0,2,0,1,1
4,0,3,0,2,0,0,1,0,0,0


In [15]:
y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis=1)
X_train = train.values # создаем вектор тренировочных данных
X_test = test.values # создаем вектор тестовых данных

In [17]:
scaler = StandardScaler() #масштабирование данных
scaler.fit(train)
train = scaler.fit_transform(train)

scaler.fit(test)
test = scaler.fit_transform(test)

In [18]:
#выбор моделей первого уровня
rfc = RandomForestClassifier() 
sgd = SGDClassifier()
log = LogisticRegression()
knc = KNeighborsClassifier()
svc = SVC()

In [19]:
#сетка параметров для GridSearchCV
rfc_params = {'n_estimators': range(250, 500, 25),
              'max_depth': range(2, 7),
              'min_samples_leaf': range(1, 4)
             }
    
sgd_params = {
              'n_iter': range(10 , 100, 5),
              'alpha': np.arange(0.00001, 0.01, 0.0001),
              'epsilon': np.arange(0.0001, 0.001),
              'eta0': np.arange(0.1, 0.5, 0.1)
             }
log_params = {'C': range(1, 10, 1)
             }
knc_params = {'n_neighbors': range(10, 20)}
    
svc_params = { 'kernel': ['linear', 'poly'],
              'C': range(1, 100, 10),
              'degree': range(1, 10)
             }

In [29]:
def gridsearch(clf, X_train, y_train, params):
    """return the best estimator"""
    searchcv = GridSearchCV(clf, param_grid=params, n_jobs=-1)
    searchcv.fit(X_train, y_train)
    return searchcv.best_estimator_

In [21]:
rfc_best = gridsearch(rfc, X_train, y_train, rfc_params)
sgd_best = gridsearch(sgd, X_train, y_train, sgd_params)
log_best = gridsearch(log, X_train, y_train, log_params) 
knc_best = gridsearch(knc, X_train, y_train, knc_params)
svc_best = gridsearch(svc, X_train, y_train, svc_params)

In [22]:
ntrain = train.shape[0] #количество образцов в тренировочном датасете
ntest = test.shape[0] #количество образцов в тестовом датасете
NFOLDS = 5 #количество фолдов для кросс валидации
kf = KFold(n_splits=NFOLDS, shuffle=False, random_state=0)


In [23]:
def get_oof(clf, X_train, y_train, X_test): 
    """out-of-fold cross validation"""
    oof_train = np.zeros((ntrain,)) #создает вектор с нулями размеров 891
    oof_test = np.zeros((ntest,)) #создает вектор с нулями размером 418
    oof_test_skf = np.empty((NFOLDS, ntest)) #создает матрицу 5 на 438

    for i, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
        x_tr = X_train[train_index]
        y_tr = y_train[train_index]
        x_te = X_train[test_index]

        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(X_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [24]:
rfc_oof_train, rfc_oof_test = get_oof(rfc_best, X_train, y_train, X_test)
sgd_oof_train, sgd_oof_test = get_oof(sgd_best, X_train, y_train, X_test)
log_oof_train, log_oof_test = get_oof(log_best, X_train, y_train, X_test)
knc_oof_train, knc_oof_test = get_oof(knc_best, X_train, y_train, X_test)
svc_oof_train, svc_oof_test = get_oof(svc_best, X_train, y_train, X_test)


In [25]:
#тренировочный и тестовый наборы для модели 2 уровня
X_train_second = np.concatenate((rfc_oof_train, sgd_oof_train, log_oof_train, knc_oof_train, svc_oof_train), axis=1) 
X_test_second = np.concatenate((rfc_oof_test, sgd_oof_test, log_oof_test, knc_oof_test, svc_oof_test), axis=1)

In [26]:
#модель второго уровня
gbm = xgb.XGBClassifier(
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(X_train_second, y_train)
predictions = gbm.predict(X_test_second)

In [28]:
submission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })

submission.to_csv("submission.csv", index=False)